# CIC_IDS_2018_preprocessed_data

In [1]:
"Cyber Security latest dataset".center(100, "*").upper()

'***********************************CYBER SECURITY LATEST DATASET************************************'

## The workflow that we will guide us

In [2]:
workflow = [ 
             "    1.   import the libraries for Exploratory Analysis",
             "    1.1. import the dataset",
             "    2.   check shape",
             "    3.   check data types",
             "    4.   check Null values",
             "    5.   Check duplicate",
             "    6.   Split data into X and y",
             "    7.   Preprocessing",
                        " 7.1.    import classifier",
                        " 7.2     import and set random seed",
                        " 7.3     import train_test_split",
                        " 7.4     fit and train ",
                        " 7.5     score and predict",
                        " 7.6     save model with pickle or joblib"
         ]


In [3]:
workflow

['    1.   import the libraries for Exploratory Analysis',
 '    1.1. import the dataset',
 '    2.   check shape',
 '    3.   check data types',
 '    4.   check Null values',
 '    5.   Check duplicate',
 '    6.   Split data into X and y',
 '    7.   Preprocessing',
 ' 7.1.    import classifier',
 ' 7.2     import and set random seed',
 ' 7.3     import train_test_split',
 ' 7.4     fit and train ',
 ' 7.5     score and predict',
 ' 7.6     save model with pickle or joblib']

In [4]:
workflow[0]

'    1.   import the libraries for Exploratory Analysis'

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import dython
import pickle
import joblib
from datetime import datetime
from dateutil import parser
from dython.nominal import associations

## Import the dataset

In [6]:
print(workflow[1])
cyber = pd.read_csv(r"C:\Users\godwi\GitHub\Canadian-Cyber-Attack-Anomaly-detection-Scikit-Learn\cic_ids_2018.csv")
cyber.head()

    1.1. import the dataset


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\godwi\\GitHub\\Canadian-Cyber-Attack-Anomaly-detection-Scikit-Learn\\cic_ids_2018.csv'

## The Shape of the dataset

In [ ]:
cyber.shape

## Info about the dataset

In [ ]:
cyber.info()

## The data types

In [ ]:
cyber.dtypes

## Checking for Null values

In [ ]:
cyber.isna().sum()

## Lets work on dropping columns that have only 0s as values , or making them NaN to drop them

In [ ]:
start = []
for col in cyber.columns:
        if col[:7] == "Bwd IAT": 
                start.append(col)
print(start)

## Filling with np.nan

In [ ]:
# lets fillna so we can remove blanck columns by dropping
cyber = cyber.replace(0, np.nan)

## Dropping the columns that have only NaNs all through

In [ ]:
cyber.dropna(how = "all", axis =1,inplace = True )
cyber.fillna(0, inplace = True)
cyber.head()

In [ ]:
cyber.dropna(how = "all", axis =0,inplace = True )
cyber.shape

## Investigating feature correlations and dropping columns with very high correlation (based on inspection of heatmap)

In [ ]:
associations(cyber, figsize=(30,30))

### Let choose columns that has a correlation of  0.2 to 0.7

corr = cyber.corr()
kot = corr[(corr == 0) ]
associations(kot.corr(),figsize=(30,30))

# let select all the columns from the heatmap
print([col for col in kot])

In [ ]:
X= cyber.drop(['Label',"Unnamed: 0"], axis =1)
y =cyber["Label"]

## Training , Fiiting 

## Train and test with seed

In [ ]:
from sklearn.model_selection import train_test_split
np.random.seed(42)
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

from sklearn import preprocessing
X_train = preprocessing.scale(X_train)
X_test = preprocessing.scale(X_test)

np.random.seed(32)
from sklearn.linear_model import SGDClassifier
sgd= SGDClassifier(loss="hinge", penalty="l2", max_iter=1000)
sgd.fit(X_train, y_train)
print(f'Training score with SGDClassifier : {  round(sgd.score(X_train,y_train)*100,2)}%')
print(f'Test accuracy score with SGDClassifier :{round(sgd.score(X_test,y_test)*100,2)}%')

np.random.seed(32)
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, n_jobs=2)
clf.fit(X_train, y_train)

print(f'Training score with RandomForestClassifier : { round(clf.score(X_train,y_train)*100,2)}%')
print(f'Test accuracy score with RandomForestClassifier :{round(clf.score(X_test,y_test)*100,2)}%')

In [ ]:
from sklearnex import patch_sklearn 


In [ ]:
patch_sklearn()

In [ ]:

from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
# from sklearn import svm
from sklearn.metrics import f1_score, recall_score, precision_score

sgd = SGDClassifier(loss="hinge", penalty="l2", max_iter=3000)
clf = RandomForestClassifier(n_estimators=100, n_jobs=2)
#svm = svm.SVC()

sgd.fit(X_train, y_train)
clf.fit(X_train, y_train)
# svm.fit(X_train, y_train)
# The shape is above 240,000
training_Score_SGD = sgd.score(X_train, y_train)
training_Score_CLF = clf.score(X_train, y_train)
# training_Score_SVM = svm.score(X_train, y_train)
test_Accuracy_Score_SGD = sgd.score(X_test, y_test)
test_Accuracy_Score_CLF = clf.score(X_test, y_test)
# test_Accuracy_Score_SVM = svm.score(X_test, y_test)

print (f'Training SGD score = {training_Score_SGD *100:.2f}%')
print (f'Test Accuracy SGD score = {test_Accuracy_Score_SGD *100:.2f}%')
print()
print (f'Training CLF score = {training_Score_CLF *100:.2f}%')
print (f'Test Accuracy CLF score = {test_Accuracy_Score_CLF *100:.2f}%')
print()
#print (f'Training SVM score = {test_Accuracy_Score_SVM *100:.2f}%')
#print (f'Test Accuracy SVM score = {test_Accuracy_Score_SVM *100:.2f}%')


## Predicting with X_test with the RandomForest clf model , which scored better than SGD classifier

In [ ]:
y_preds = clf.predict(X_test)
y_preds[:20]

## The evaluation Matrices

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

accuracy_score(y_test, y_preds)

### Classification Report

In [ ]:
print(classification_report(y_test, y_preds))

In [ ]:
cm = confusion_matrix(y_test, y_preds)
cm

## making a dataframe of the confusion matrix

In [ ]:
cmd = pd.DataFrame(cm, columns = (['Benign', 'Bot', 'DDOS attack-HOIC', 'DDOS attack-LOIC-UDP',
       'DoS attacks-GoldenEye',"DoS attacks-Hulk","DoS attacks-SlowHTTPTest","DoS attacks-Slowloris","FTP-BruteForce"\
              ,"Infilteration","SSH-Bruteforce"]), index =(['Benign', 'Bot', 'DDOS attack-HOIC', 'DDOS attack-LOIC-UDP',
       'DoS attacks-GoldenEye',"DoS attacks-Hulk","DoS attacks-SlowHTTPTest","DoS attacks-Slowloris","FTP-BruteForce"\
              ,"Infilteration","SSH-Bruteforce"] ))
cmd 

In [ ]:
fig, ax = plt.subplots(figsize = (19,6))
sns.heatmap(cmd/np.sum(cmd), fmt= '.1%' ,linewidth=0.3, cmap = 'Greens',xticklabels = True, annot =True)
ax.set(title = 'Confusion Matrix', xlabel = 'Predicted Class', ylabel = 'Actual Class');

## Saving the model with joblib


In [ ]:
import joblib
print(f"Model is now saved !!".center(50,"*"))
print(f' Saving the predictive models as: {joblib.dump(clf, "model.joblib")}')
print(f"Model is now saved !!".center(50,"*"))